In [1]:
from numpy import array
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding

/home/joydeb/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/joydeb/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/joydeb/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/joydeb/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [ ]:
class Preprocessing():
    def __init__():
        self.file = file
    
    def sentence_from_file(self):
    f = open(self.filename)
    single_file_sentences = []
    sentence_list = []
    for line in f:
        if len(line)==0 or line[0]=="\n":
            if len(sentence_list) > 0:
                single_file_sentences.append(sentence_list)
                sentence_list = []
            continue
        splits = line.split(' ')
        sentence_list.append([splits[0],splits[1],splits[-1]])

    if len(sentence_list) >0:
        single_file_sentences.append(sentence_list)
        sentence_list = []
    return single_file_sentences

    def get_case_value(self,word, case_dict):   
        case_value = 'other'

        count_digits = 0
        for char in word:
            if char.isdigit():
                count_digits += 1

        if word.isdigit():
            case_value = 'number'
        elif count_digits / float(len(word)) > 0.5:
            case_value = 'fraction'
        elif word.islower():
            case_value = 'lower'
        elif word.isupper():
            case_value = 'upper'
        elif word[0].isupper():
            case_value = 'title'
        elif count_digits > 0:
            case_value = 'leters_digit'

        return case_dict[case_value]


    def createBatches(self,data):
        l = []
        for i in data:
            l.append(len(i[0]))
        l = set(l)
        batches = []
        batch_len = []
        z = 0
        for i in l:
            for batch in data:
                if len(batch[0]) == i:
                    batches.append(batch)
                    z += 1
            batch_len.append(z)
        return batches,batch_len

    def createMatrices(self,sentences, wd_to_id, lb_to_id, cs_to_id, ch_to_id, pos_to_id):
        #paddingIdx = word2Idx['PAD_TKN']
        unknownIdx = wd_to_id['UNK_TKN']

        dataset = []

        word_count = 0
        unknownword_count = 0

        for sentence in sentences:
            word_indices = []    
            char_indices = []
            case_indices = []
            label_indices = []
            pos_indices = []

            for word,char,pos,label in sentence:  

                word_count += 1
                if word in wd_to_id:
                    word_index = wd_to_id[word]
                elif word.lower() in wd_to_id:
                    word_index = wd_to_id[word.lower()]                 
                else:
                    word_index = unknownIdx
                    unknownword_count += 1
                char_index = []
                for x in char:
                    char_index.append(ch_to_id[x])
                word_indices.append(word_index)
                case_indices.append(get_case_value(word, cs_to_id))
                pos_indices.append(pos_to_id[pos.replace('\n','')])
                char_indices.append(char_index)
                label_indices.append(lb_to_id[label])

            dataset.append([word_indices, case_indices, char_indices, pos_indices, label_indices]) 
        return dataset

    def iterate_minibatches(self,dataset,batch_len): 
        start = 0
        for i in batch_len:
            tokens = []
            char = []
            labels = []
            casing = []
            pos_tags = []
            data = dataset[start:i]
            start = i
            for dt in data:
                t,c,ch,pos,l = dt
                l = np.expand_dims(l,-1)
                tokens.append(t)
                char.append(ch)
                labels.append(l)
                casing.append(c)
                pos_tags.append(pos)
            yield np.asarray(labels),np.asarray(tokens),np.asarray(casing), np.asarray(char), np.asarray(pos_tags)

    def addCharInformatioin(self,Sentences):
        for i,sentence in enumerate(Sentences):
            for j,data in enumerate(sentence):
                chars = [c for c in data[0]]
                Sentences[i][j] = [data[0],chars,data[1],data[2]]
        return Sentences

    def padding(self,Sentences):
        maxlen = 52
        for sentence in Sentences:
            char = sentence[2]
            for x in char:
                maxlen = max(maxlen,len(x))
        for i,sentence in enumerate(Sentences):
            Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
        return Sentences
